In [275]:
import os
import numpy as np
import csv

We need to get from every sample:
 - UVW and XY (.new)
 - kalpha 1 and kalpha 2  (.xrdml)
 - 2theta (start and end position) (.xrdml)
 - counts (.xrdml)
 - background (start, step, end) (.bac)

In [276]:
def files_names(path):
    '''Function to see structure of data in files and catalogs'''
    with open("names.txt", "w", encoding="utf-8") as names_f:
        for root, dirs, files in os.walk(path):
            names_f.write(f"[PATH]{root}\n")
            for d in dirs: 
                names_f.write(f"[DIR]{d}\n")
            for f in files: 
                names_f.write(f"[FILE]{f}\n")
            names_f.write("\n")


In [277]:
def document_names(extention,ext_file):
    '''Function separating files with specific extention'''
    new = open(ext_file,'w')
    with open("names.txt",'r') as names_f:
        for line in names_f:
            if '[PATH]' in line:
                new.write(f"{line}")
            elif '[FILE]' and extention in line:
                    new.write(f"{line}")
        new.write(f"\n")

In [278]:
def generate_document(ext,ext_file):
    document_names(ext,ext_file)
    with open(ext_file,'r') as new_file:
        file = new_file.read().replace("\\","/")
    with open(ext_file, "w", encoding="utf-8") as new_file:
        new_file.write(file) 

In [279]:
def read_new_doc(ext_file):
    '''function saving paths and file names in order in variables'''
    path = []
    files_names = []
    with open(ext_file,'r') as new_doc:
        lines = new_doc.readlines()
        for i,line in enumerate(lines):
            if '[PATH]' in line:
                clean_path = line.replace("[PATH]","").strip()
                n = 1
                if i+n< len(lines) and '[FILE]' in lines[i+n]:
                    while '[FILE]' in lines[i+n]:
                        clean_file = lines[i+n].replace("[FILE]","/").strip()
                        full_path = clean_path+clean_file
                        path.append(full_path)
                        file_name = lines[i+n].replace("[FILE]","").strip()
                        files_names.append(file_name)
                        n=n+1
    return path,files_names

In [280]:
def read_inside(path):
    file = open(str(path),'r')
    read_content = file.read()
    print(read_content)
    file.close()

In [281]:
def find_xrdml_file(path,name,only_name):
    '''saving data from files to variables:
        kalpha_theta = ['kAlpha1','kAlpha2','2Theta - start position','2Theta - end position']
        counts = ['all the counts']'''
    base, _ = os.path.splitext(path)
    new_path = base + ".xrdml"
    # read_inside(new_path)
    if not os.path.exists(new_path):
        return False, False 
    kalpha_theta = []
    counts = []
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if '<kAlpha1 unit="Angstrom">' in line:
                kalpha_theta.append(float(line.split(">")[1].split("<")[0]))
            if '<kAlpha2 unit="Angstrom">' in line:
                kalpha_theta.append(float(line.split(">")[1].split("<")[0]))
            if '<positions axis="2Theta" unit="deg">' in line:
                kalpha_theta.append(float(numered_lines[i+1].split(">")[1].split("<")[0]))
                kalpha_theta.append(float(numered_lines[i+2].split(">")[1].split("<")[0]))
            if 'counts'in line:
                count = line.split(">")[1].split("<")[0]
                counts = count.split()
    for i in range(len(counts)):
        counts[i]=int(counts[i])      
    # print(kalpha_theta)
    # print(counts)
    return kalpha_theta,counts

In [282]:
def find_bac_file(path,name,only_name):
    '''saving data from background to variables:
        start_step_end = ['start','step','end']
        counts_bac = ['all the counts from background']'''
    base, _ = os.path.splitext(path)
    new_path = base + ".bac"
    # read_inside(new_path)
    if not os.path.exists(new_path):
        return False, False 
    start_step_end = []
    counts_bac = []
    with open(new_path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        for i,line in enumerate(numered_lines):
            if i == 0:
                a = line.split()
                for n in range(3):
                    start_step_end.append(float(a[n]))
            if i != 0:
                counts_b = line.split()
                for n in range(len(counts_b)):
                    counts_bac.append(int(counts_b[n]))
    # print(counts_bac)
    return start_step_end,counts_bac

In [283]:
def doc_new_inside(path,name):
    '''function saving uvw and xy
        uvw_xy = ['U','V','W','X','Y','GauSiz','LorSiz','Size-Model']
        and using other function to create a file with all the most important data'''
    index = []
    uvw_xy =[]
    uvw_xy_hedlines =[]
    with open(path,'r') as new_doc:
        numered_lines = new_doc.readlines()
        # name_of_sample = numered_lines[0]
        for i,line in enumerate(numered_lines):
            if line.strip() == '!':
                continue
            if '!' in line:
                if i>2:
                    index.append(i)  
        if numered_lines[-2] != '!  2Th1/TOF1    2Th2/TOF2  Pattern to plot\n':
                    index.append(0)  
        # print(index)
        # print(path)
        fourth_from_end = index[-4]
        # uvw_xy_hedlines = numered_lines[fourth_from_end].lstrip("!").split()
        uvw_xy = numered_lines[fourth_from_end+1].split()
    for i in range(len(uvw_xy)):
        uvw_xy[i]=float(uvw_xy[i])
    # print(uvw_xy)
    # print(uvw_xy_hedlines)
    only_name, _ = os.path.splitext(name)

    kalpha_theta,counts = find_xrdml_file(path,name,only_name)
    start_step_end,counts_bac = find_bac_file(path,name,only_name)
    
    with open('data_storage.txt', 'a') as data_storage:
        # data_storage.write('')
        data_storage.write('[PATH]')
        data_storage.write(path)
        data_storage.write('\n')
        data_storage.write(only_name)
        data_storage.write('\n')
        # data_storage.write(uvw_xy_hedlines)
        data_storage.write(str(uvw_xy))
        data_storage.write('\n')
        if kalpha_theta == False:
            data_storage.write('[NO FILE]\n')
            data_storage.write('[NO FILE]\n')
        else:
            data_storage.write(str(kalpha_theta))
            data_storage.write('\n')
            data_storage.write(str(counts))
            data_storage.write('\n')
        if start_step_end == False:
            data_storage.write('[NO FILE]\n')
            data_storage.write('[NO FILE]\n')
        else:
            data_storage.write(str(start_step_end))
            data_storage.write('\n')
            data_storage.write(str(counts_bac))
            data_storage.write('\n')
            

In [284]:
path = r"D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane"
files_names(path)

extentions = ['.new','.xrdml','.bac']
extentions_files = ['new_files.txt','xrdml_files.txt','bac_files.txt']
all_paths =[]
all_names = []
for i in range(len(extentions)):
    generate_document(extentions[i],extentions_files[i])
    paths,names = read_new_doc(extentions_files[i]) 
    all_paths.append(paths)
    all_names.append(names) 
    
length_names = len(all_names[0][:])           
length_paths = len(all_paths)   

# print(all_paths)
# print(all_names)
# print(all_names[0])
# print(all_names[1])
# print(all_names[2])
with open('data_storage.txt', 'w') as data_storage:
    data_storage.write('')
for i in range(length_names):
    doc_new_inside(all_paths[0][i],all_names[0][i] )





In [285]:
# def doc_new_inside(path,name):
#     '''function saving uvw and xy'''
#     inside = []
#     index = []
#     headline = []
#     uvw_xy =[]
#     uvw_xy_hedlines =[]
#     with open(path,'r') as new_doc:
#         inside.append(path)
#         numered_lines = new_doc.readlines()
#         inside.append(numered_lines[0].strip())
#         for i,line in enumerate(numered_lines):
#             if line.strip() == '!':
#                 continue
#             if '!' in line:
#                 if i>2:
#                     index.append(i)
#                 a = line.replace("!","").strip()
#                 if i+1< len(numered_lines) and '!' in numered_lines[i+1]:
#                     inside.append(a)
#                 elif i+1< len(numered_lines):
#                     inside.append(a)
#                     n=1
#                     while i+n<len(numered_lines) and '!' not in numered_lines[i+n]:
#                         b = numered_lines[i+n].strip()
#                         c=''+b
#                         n=n+1
#                     inside.append(c)
#         print(inside)
#         print(index)
#         fourth_from_end = index[-4]
#         for i,line in enumerate(numered_lines):
#             if i in index :
#                 h_line = line.lstrip("!").strip()
#                 headline.append(h_line)
#         uvw_xy_hedlines = numered_lines[fourth_from_end].split()
#         uvw_xy = numered_lines[fourth_from_end+1].split()
#     print(uvw_xy)
#     print(headline)
#     with open("test.txt", "w", encoding="utf-8") as test:
#         test.write(str(inside))  
#     only_name, _ = os.path.splitext(name)
#     csv_name = only_name + ".csv"
#     with open(csv_name, "w", newline="", encoding="utf-8") as csv_file:
#         writer = csv.writer(csv_file)



# doc_new_inside(all_paths[0],all_names[0])


In [286]:
# def all_data_files_of_sample(path,name):
#     with open(path, "r", encoding="utf-8") as txt_file:
#         lines = txt_file.readlines()
#     only_name, _ = os.path.splitext(name)
#     csv_name = only_name + ".csv"
#     with open('names.txt', "r", encoding="utf-8") as base:

    

In [287]:
# import csv
# def save_to_csv(path,name):
#     with open(path, "r", encoding="utf-8") as txt_file:
#         lines = txt_file.readlines()
#     only_name, _ = os.path.splitext(name)
#     csv_name = only_name + ".csv"
#     with open(csv_name, "w", newline="", encoding="utf-8") as csv_file:
#         writer = csv.writer(csv_file)
#         hedlines = ['Title','Current global Chi2 (Bragg contrib.)','Files names']
#         lines[0].strip()
#         # written = False
#         # for line in lines:
#         #     line = line.strip()
#         #     if not line:
#         #         continue

#         #     if line.startswith("!") and not written:
#         #         headline = line.lstrip("!").strip()
#         #         columns = headline.split()
#         #         writer.writerow(columns)
#         #         written = True
#         #         continue

#         #     if not line.startswith("!"):
#         #         parts = line.split()
#         #         if all(c.replace(".", "", 1).replace("-", "", 1).isdigit() for c in parts):
#         #             writer.writerow(parts)

# save_to_csv(all_paths[0],all_names[0])